[View in Colaboratory](https://colab.research.google.com/github/geneing/tacotron-1/blob/tacotron2-work-in-progress/vocoder_dev.ipynb)

In [5]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip3 install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision

import torch
print(torch.__version__)
#print(torch.cuda.current_device())
print(torch.cuda.get_device_name(0))

Found GPU at: /device:GPU:0
    100% |████████████████████████████████| 484.0MB 50.4MB/s 
tcmalloc: large alloc 1073750016 bytes == 0x5b5a6000 @  0x7f3fea8101c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8
Requirement already up-to-date: torchvision in /usr/local/lib/python3.6/dist-packages (0.2.1)
Requirement not upgraded as not directly required: numpy in /usr/local/lib/python3.6/dist-packages (from torchvision) (1.14.3)
Requirement not upgraded as not directly required: six in /usr/local/lib/python3.6/dist-packages (from torchvision) (1.11.0)
Requirement not upgraded as not directly required: pillow>=4.1.1 in /usr/local/lib/python3.6/dist-packages (from torchvision) (5.1.0)
  Found existing installation: torch 0.4.0
    Uninstalling torch-0.4.0:
      Successful

Upload full LJ dataset to google drive. Need to do this only once. Disabled by using magic %%script.

In [0]:
!pip3 install -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



In [0]:
%%script false
!curl -O -L "http://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2" 
fid = '170SC-OItkxCU536ikdRTDLW96z3Ho7CZ' #Collab Notebooks/Data 
upload = drive.CreateFile({"parents": [{"kind": "drive#fileLink", "id": fid}], 'title': 'LJSpeech-1.1.tar.bz2'})
upload.SetContentFile('LJSpeech-1.1.tar.bz2')
upload.Upload()


Download Speech data.

In [0]:

def prep_archive(fid, fname):
  import os
  if not os.path.isfile('data/%s'%fname):  
    dld = drive.CreateFile({'id': fid})
    dld.GetContentFile(fname)
    !mkdir -p data/LJSpeech
    !mv {fname} data/

  if not os.path.isdir('data/LJSpeech-1.1'): #not quite correct
    !tar --strip-components 1 -xjf  data/{fname} -C data/LJSpeech
  

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
#prep_archive(fid = '1EJdkZM2vhRy80ReHDwrbUB8wfsVJHBzW', fname='LJSpeech-1.1.tar.bz2') #full archive
prep_archive(fid = '1DmkZ7rDIYf_JZiBZlVOvnXGxuSMTYqVv', fname='LJSpeech-1.1.tar.bz2') #100 wavs



In [91]:
!git clone https://github.com/geneing/tacotron-1.git tacotron
!cd tacotron; git checkout tacotron2-work-in-progress; git pull; pip3 install -r requirements.txt
!python3 tacotron/preprocess.py --dataset ljspeech --base_dir data/LJSpeech --output ../LJSpeech.taco



fatal: destination path 'tacotron' already exists and is not an empty directory.
Already on 'tacotron2-work-in-progress'
Your branch is up-to-date with 'origin/tacotron2-work-in-progress'.
Already up-to-date.
/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
 28%|████████████▏                              | 28/99 [00:09<00:23,  2.97it/s]

100%|███████████████████████████████████████████| 99/99 [00:29<00:00,  3.36it/s]
Wrote 99 utterances, 52349 frames (0.18 hours)
Max input length:  198906
Max output length: 796


In [92]:
!ls /content/data/



LJSpeech  LJSpeech-1.1.tar.bz2	LJSpeech.taco


# Now the actual work begins

In [0]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np

class SpeechDataset(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.metadata = pd.read_csv(os.path.join(self.root_dir, "train.txt"), sep='|', 
                                    header=None, names=['spec','mel','wav','mel_len','wav_len','text'])
        
    def __len__(self):
        return len(self.metadata)
    
    def __getitem__(self, idx):
        sample = {}
        wav_path=os.path.join(self.root_dir, self.metadata.wav[idx])
        sample['text'] = self.metadata.text[idx]
        sample['wav'] = np.load(wav_path)
        return sample

root_dir='data/LJSpeech.taco'
ds = SpeechDataset(root_dir)


In [0]:
quant = 256
hidden_size = 896
eye = np.eye(quant,dtype=np.float32)
def mergeSamples(inp):
    wavlist=[]
    strlist=[]
    for v in inp:
        quantized = (((v['wav']+1.)/2.)*quant).astype(int)
        
        wavlist.append(quantized)
        strlist.append(v['text'])
    maxlen = max([x.shape[0] for x in wavlist])
    
    wavlist = np.asarray([  np.pad(x, ((0,maxlen-x.shape[0]),), 'constant') for x in wavlist])
    return {'wav':wavlist.T, 'text':strlist}

dl = DataLoader( ds, batch_size = 4, shuffle=True, collate_fn = mergeSamples )


In [105]:
gru = torch.nn.GRUCell(quant, hidden_size).cuda()
O1 = torch.nn.Linear(hidden_size, hidden_size//2).cuda()
relu = torch.nn.ReLU().cuda()
O2 = torch.nn.Linear(hidden_size//2, quant).cuda()

params=[]
for p in [gru, O1, O2]:
    params += list(p.parameters()) 

optimizer = torch.optim.Adam(params, lr = 0.0001)

for sample in dl:
    wav = sample['wav']
    h_rnn = torch.zeros([wav.shape[1], hidden_size]).cuda()
    optimizer.zero_grad()
    loss = 0
    for i in range(min([1000,wav.shape[0]])):
        coded_wav = torch.tensor(eye[ wav[i, :] ]).cuda()
        h_rnn = gru(coded_wav, h_rnn)
        o1 = relu(O1(h_rnn))
        o2 = O2(o1)    
        
        loss += torch.nn.functional.cross_entropy(o2, torch.tensor(wav[i+1, :]).cuda())
    print(loss)
    loss.backward()
    optimizer.step()
    #break


RuntimeError: ignored

In [108]:
torch.cuda.memory_cached()

318242816